In [1]:
import aocd
import re
import numpy as np
import networkx as nx
%run helper.ipynb
puzzle = aocd.models.Puzzle(year=2024, day=16)
data = puzzle.input_data

In [2]:
grid = Grid(data, str)

In [3]:
turning_dict = {
    Point2D((0,1)): [Point2D((1,0)), Point2D((-1,0))],
    Point2D((0,-1)): [Point2D((1,0)), Point2D((-1,0))],
    Point2D((1,0)): [Point2D((0,1)), Point2D((0,-1))],
    Point2D((-1,0)): [Point2D((0,1)), Point2D((0,-1))],
}

start = Point2D(np.transpose(np.where(grid == 'S'))[0])
end = Point2D(np.transpose(np.where(grid == 'E'))[0])
facing = Point2D((0,1))
position = (start, facing)

def get_valid_adj(position, grid):
    valid = []
    turn1 = position[0] + turning_dict[position[1]][0]
    if grid.in_bounds(turn1) and grid[*(turn1)] != "#":
        valid.append(((turn1, turning_dict[position[1]][0]), 1001))
    turn2 = position[0] + turning_dict[position[1]][1]
    if grid.in_bounds(turn2) and grid[*(turn2)] != "#":
        valid.append(((turn2, turning_dict[position[1]][1]), 1001))
    no_turn = position[0] + position[1]
    if grid.in_bounds(no_turn) and grid[*(no_turn)] != "#":
        valid.append(((no_turn, position[1]), 1))
    return valid

In [4]:
def build_graph(position, graph, grid):
    G.add_node(position)
    adj = get_valid_adj(position, grid)
    next_adj = [a for a in adj if not graph.has_node(a[0])]
    for a in adj:
        G.add_edge(position, a[0], weight=a[1])
    for a in next_adj:
        build_graph(a[0], graph, grid)


In [5]:
G = nx.Graph()
build_graph(position, G, grid)
for k in turning_dict:
    G.add_edge((end, k), "true_end", weight=0)

In [6]:
dist = nx.shortest_path_length(G, (start, facing), "true_end", weight='weight')

In [7]:
puzzle.answer_a = dist

In [9]:
tile_set = set()
asp = nx.all_shortest_paths(G, (start, facing), "true_end", weight='weight')
for p in asp:
    for t in p:
        if t != 'true_end':
            tile_set.add(t[0])

In [10]:
puzzle.answer_b = len(tile_set)